In [1]:
import os
import requests
import json

In [2]:
TWITTER_TOKEN = os.getenv('TWITTER_TOKEN')

In [3]:
def get_twitter_id_from_username(username):
    url = f'https://api.twitter.com/2/users/by/username/{username}'
    header = { 'Authorization': f'Bearer {TWITTER_TOKEN}'}
    response = requests.get(url, headers=header)
    
    if response.status_code == 200:
        return response.json()['data']
    else:
        print('error on request')

In [4]:
twitter_user = get_twitter_id_from_username('nelth_fr'); twitter_user

{'id': '996727069798264832', 'name': 'Florent HAFFNER', 'username': 'nelth_fr'}

In [5]:
def get_twitter_data_from_userid(user_id, max_results, next_token):
    url = f'https://api.twitter.com/2/users/{user_id}/tweets'
    url_params = '?'+ \
        f'max_results={max_results}&' + \
        'tweet.fields=created_at&' + \
        'expansions=author_id'
    if next_token:
        url_params += f'&pagination_token={next_token}'

    header = { 'Authorization': f'Bearer {TWITTER_TOKEN}'}   
    response = requests.get(url+url_params, headers=header)
    
    if response.status_code == 200:
        return response.json()
    else:
        print('error on request')

In [6]:
user_id = twitter_user['id']; user_id

'996727069798264832'

In [7]:
def get_full_tweets(max_results=500, nbr_per_page=100):
    dataset = []
    
    try:
        results = get_twitter_data_from_userid(user_id, nbr_per_page, '')
        next_token = results['meta']['next_token']
        for res in results['data']:
            dataset.append(res)

        nbr_pages = int(max_results/nbr_per_page)
        for i in range(0, nbr_pages):
            results = get_twitter_data_from_userid(user_id, nbr_per_page, next_token)
            next_token = results['meta']['next_token']
            for res in results['data']:
                dataset.append(res)
    
    except KeyError:
        print('seems like you don\'t have access to more tweets...')

    return dataset

In [8]:
archive = get_full_tweets()

seems like you don't have access to more tweets...


In [9]:
len(archive)

300

In [15]:
def write_data_to_fs(data):
    directory_to_write = '../../data'
    if not os.path.exists(directory_to_write):
        os.mkdir(directory_to_write)
    with open(f'{directory_to_write}/tweets-{user_id}.json', 'w') as outfile:
        json.dump(data, outfile)

In [16]:
write_data_to_fs(archive)